In [12]:
"""
Takes [hml_filename].hml
and converts them into csv format

Beware of overwriting
Choose the file name (source and dest) carefully

Beware of duplicated / missing / combined data records
(especially on appended records in the same file)
In hml_to_csv, the default is to convert the last record on the file

Beware of the enabled metrics, keep them all the same throughout the file
Multiple records with diferrent metrics WILL conflict

The code will try to read the GPU name from the log file,
overwrite the name if needed

Also edit app name

Errors -> Good luck
"""

import csv
import datetime
from itertools import islice
from pathlib import Path

# Edit the path and filenames
log_path = "C:/Users/wafffle/Downloads/NSO/Game-Benchmark-Driver-Performance-Evaluation-/test_sources/"
hml_filename = "HardwareMonitoring"

# Overwrite the columns in csv if not empty
# Edit app_name if the .exe file name is as clear as mud to the reader
gpu_name = ""
app_name = "Wuthering Waves"


In [ ]:
def hml_to_csv(src, dest, gpu_name='', app_name='', convert_at_records: int|list[int]|range|str=-1, encoder='cp1252'):

    hml_path = Path(src).with_suffix('.hml')
    if not hml_path.exists():
        raise FileNotFoundError()
    
    print(f"Received: {hml_path}")
    start_record_rows = []
    with open(hml_path, 'r', newline='', encoding=encoder) as infile:
        # Read the file twice, one for finding the starting line numbers of the records,
        # and then actually write the data.
        reader = csv.reader(infile)
        for rownumber, row in enumerate(reader):
            if row and (row[0] == '01'):
                start_record_rows.append(rownumber)

    began = False
    convert_start_rows = []
    if isinstance(convert_at_records, int):
        convert_start_rows.append(start_record_rows[convert_at_records])
    elif isinstance(convert_at_records, str) and convert_at_records == 'all':
        convert_start_rows = start_record_rows
    else:
        for rownumber in iter(convert_at_records):
            convert_start_rows.append(start_record_rows[rownumber])

    use_file_index = False
    if len(convert_start_rows) > 1:
        use_file_index = True
    
    print(f"Found {len(start_record_rows)} records.")
    print(f"Converting to {len(convert_start_rows)} file(s)...")
    print()

    for convert_number, convert_start_row in enumerate(convert_start_rows):
        began = False
        data_rows = []
        headers = []
        units = []

        # start reading from ...
        with open(hml_path, 'r', newline='', encoding=encoder) as infile:
            reader = csv.reader(infile)
            for row in islice(reader, convert_start_row, None):
                if row[0] == '01':
                    # Extract GPU name (first GPU if multiple)
                    if not began:
                        began = True
                        if not gpu_name:
                            gpu_name = row[2].split(',')[0].strip()
                    else:
                        # Hit a different record
                        break
                elif row[0] == '02':
                    headers = row[2:]
                elif row[0] == '03':
                    units.append(row[3].strip())
                elif row[0] == '80':
                    data_rows.append(row[1:])


        combined_headers = ["GPU", "Application", "Timestamp"] + [f"{h.strip()} ({u})" if u else h.strip() for h, u in zip(headers, units)]
        this_result_path = dest
        if use_file_index:
            this_result_path += f'_{convert_number}'
        this_result_path = Path(this_result_path).with_suffix('.csv')

        with open(this_result_path, 'w', newline='', encoding=encoder) as outfile:
            writer = csv.writer(outfile)
            writer.writerow(combined_headers)
            for row in data_rows:
                writer.writerow([cell.strip() for cell in [gpu_name, app_name] + row])

        
        dtformat = r"%d-%m-%Y %H:%M:%S"
        start_time = datetime.datetime.strptime(data_rows[0][0].strip(), dtformat)
        end_time = datetime.datetime.strptime(data_rows[-1][0].strip(), dtformat)
        
        print(f"Record #{convert_number} | Starting at: {start_time}")
        print(f"Converted to: {this_result_path}")
        print(f"Record time: {end_time-start_time} | Total lines converted: {len(data_rows)}")
        print()


In [22]:
dest_path = "C:/Users/wafffle/Downloads/NSO/Game-Benchmark-Driver-Performance-Evaluation-/results/"
dest_filename = "HardwareMonitoring"

hml_to_csv(
    src= log_path + hml_filename,
    dest = dest_path + dest_filename,
    gpu_name=gpu_name,
    app_name=app_name,
    convert_at_records='all') 


Received: C:\Users\wafffle\Downloads\NSO\Game-Benchmark-Driver-Performance-Evaluation-\test_sources\HardwareMonitoring.hml
Found 3 records.
Converting to 3 file(s)...

Record #0 | at 2026-01-03 17:43:56
Converted to: C:\Users\wafffle\Downloads\NSO\Game-Benchmark-Driver-Performance-Evaluation-\results\HardwareMonitoring_0.csv
Record time: 0:05:13 | Total lines converted: 313

Record #1 | at 2026-01-03 21:32:22
Converted to: C:\Users\wafffle\Downloads\NSO\Game-Benchmark-Driver-Performance-Evaluation-\results\HardwareMonitoring_1.csv
Record time: 0:00:44 | Total lines converted: 45

Record #2 | at 2026-01-03 21:33:22
Converted to: C:\Users\wafffle\Downloads\NSO\Game-Benchmark-Driver-Performance-Evaluation-\results\HardwareMonitoring_2.csv
Record time: 0:00:14 | Total lines converted: 15



In [25]:
# # Path to the HML file
# hml_file = Path(log_path + hml_filename).with_suffix('.hml')
# csv_file = Path(log_path + hml_filename).with_suffix('.csv')

# # Path to the benchmark file
# benchmark_file = Path(log_path + txt_filename).with_suffix('.txt')
# benchmark_csv = Path(log_path + txt_filename).with_suffix('.csv')

# # Check if files exist
# if not hml_file.exists():
#     print(f"{hml_file} not found, cannot convert to CSV")
# if not benchmark_file.exists():
#     print(f"{benchmark_file} not found, cannot convert to CSV")

# # Read the HML file and write to CSV
# headers = []
# units = []
# data_rows = []

# # Get application from benchmark
# if benchmark_file.exists():
#     with open(benchmark_file, 'r', encoding=encoder) as f:
#         for line in f:
#             if 'benchmark completed' in line:
#                 parts = line.split(',')
#                 time_app = parts[1].strip()
#                 time, app_part = time_app.split(' ', 1)
#                 if not app_name:
#                     app_name = app_part.split(' ')[0]
#                 break

# if hml_file.exists():
#     with open(hml_file, 'r', newline='', encoding=encoder) as infile:
#         reader = csv.reader(infile)
#         for row in reader:
#             if row and row[0] == '01':
#                 # Extract GPU name (first GPU if multiple)
#                 if not gpu_name:
#                     gpu_name = row[2].split(',')[0].strip()
#             elif row and row[0] == '02':
#                 headers = row[2:]  # Skip type and timestamp
#             elif row and row[0] == '03':
#                 units.append(row[3].strip())  # Unit is in column 3
#             elif row and row[0] == '80':
#                 data_rows.append(row[1:])  # Timestamp and values

# # Combine headers with units
# combined_headers = ["GPU", "Application", "Timestamp"] + [f"{h.strip()} ({u})" if u else h.strip() for h, u in zip(headers, units)]

# # Write to CSV if data exists
# if data_rows:
#     with open(csv_file, 'w', newline='', encoding=encoder) as outfile:
#         writer = csv.writer(outfile)
#         writer.writerow(combined_headers)
#         for row in data_rows:
#             writer.writerow([gpu_name, app_name] + row)
#     print(f"Converted {hml_file} to {csv_file}")
# else:
#     print(f"No data to convert from {hml_file}")

# # Now process benchmark.txt
# benchmark_data = []

# if benchmark_file.exists():
#     with open(benchmark_file, 'r', encoding=encoder) as f:
#         lines = f.readlines()

#     i = 0
#     while i < len(lines):
#         line = lines[i].strip()
#         if 'benchmark completed' in line:
#             # Parse timestamp and app
#             parts = line.split(',')
#             date = parts[0].strip()
#             time_app = parts[1].strip()
#             time, app_parsed = time_app.split(' ', 1)
#             app_parsed = app_parsed.split(' ')[0]  # Client-Win64-Shipping.exe
#             if app_name:
#                 app_parsed = app_name
#             timestamp = f"{date} {time}"
#             # Next 5 lines are metrics
#             metrics = []
#             for j in range(1, 6):
#                 if i + j < len(lines):
#                     metric_line = lines[i + j].strip()
#                     if ':' in metric_line:
#                         value = metric_line.split(':')[1].strip().split(' ')[0]
#                         metrics.append(value)
#             if len(metrics) == 5:
#                 benchmark_data.append([gpu_name, app_parsed, timestamp] + metrics)
#             i += 6  # Skip the 5 metric lines
#         else:
#             i += 1

# # Write benchmark CSV if data exists
# if benchmark_data:
#     benchmark_headers = ['GPU', 'Application', 'Timestamp', 'Average Framerate', 'Minimum Framerate', 'Maximum Framerate', '1% Low Framerate', '0.1% Low Framerate']
#     with open(benchmark_csv, 'w', newline='', encoding=encoder) as outfile:
#         writer = csv.writer(outfile)
#         writer.writerow(benchmark_headers)
#         writer.writerows(benchmark_data)
#     print(f"Converted {benchmark_file} to {benchmark_csv}")
# else:
#     print(f"No data to convert from {benchmark_file}")